In [1]:
# imports
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import numpy as np
import sys, os, os.path
import subprocess
import time
import random
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import skimage.io as sio
import skimage.transform as sm

from models import *
import tools

In [2]:
tiny_nfc_net.__file__

'/esat/opal/kkelchte/docker_home/tensorflow/pytorch_pilot/pilot/models/tiny_nfc_net.py'

In [2]:
n_frames=3
######## choose device
device='cpu'
# device='cuda:0'

######## choose model
network='tinyv3_net'
net = eval(network).Net(3).to(device)

# network='tiny_3d_net'
# network='tiny_nfc_net'
# net = eval(network).Net(3, n_frames=n_frames).to(device)

# network='tiny_LSTM_net'
# net = eval(network).Net(3).to(device)

######## initialize randomly
for p in net.parameters():
    nn.init.normal_(p,0, 0.1)

In [3]:
net

Net(
  (network): TinyNet(
    (features): Sequential(
      (0): Conv2d(3, 10, kernel_size=(10, 10), stride=(4, 4))
      (1): ReLU(inplace)
      (2): Conv2d(10, 20, kernel_size=(5, 5), stride=(4, 4))
      (3): ReLU(inplace)
    )
    (classifier): Sequential(
      (0): Linear(in_features=980, out_features=512, bias=True)
      (1): ReLU(inplace)
      (2): Linear(in_features=512, out_features=3, bias=False)
    )
  )
)

In [4]:
# list trainable variables + GPU size + feature layers + decision layers
params=0
for p in net.parameters():
#     print p.shape
    p_values=1
    for d in list(p.shape[:]):
        p_values*=d
    params+=p_values
    
feature_layers, decision_layers=0,0
# for l in net.features:
#     if 'Conv' in str(l) or 'Linear' in str(l): 
#         feature_layers+=1
# for l in net.network.classifier:
#     if 'Conv' in str(l) or 'Linear' in str(l): 
#         decision_layers+=1
print("feature layers: {0}, decision layers: {1}, parameters: {2}".format(feature_layers, 
                                                                          decision_layers, 
                                                                          params))
print("\n")
nvidia=subprocess.check_output(["nvidia-smi"])
print nvidia

# print net

feature layers: 0, decision layers: 0, parameters: 511838


Wed Apr 17 11:29:53 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    Off  | 00000000:01:00.0  On |                  N/A |
| 50%   82C    P2   130W / 250W |   2154MiB / 12187MiB |     70%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                

In [3]:
# load one run of esatv3
run_dir='/esat/opal/kkelchte/docker_home/pilot_data/esatv3_expert/00000_esatv3/RGB'
images=[run_dir+'/'+im for im in sorted(os.listdir(run_dir)) if im.endswith('jpg')]
data=[]
im_size=net.default_image_size
for img_file in images[:100]:
#     img = sio.imread(img_file)    
#     img = np.swapaxes(img,1,2)
#     img = np.swapaxes(img,0,1)
#     scale_height = int(np.floor(img.shape[1]/im_size[1]))
#     scale_width = int(np.floor(img.shape[2]/im_size[2]))
#     img = img[:,::scale_height,::scale_width]
#     img = sm.resize(img,im_size,mode='constant').astype(np.float16)
#     img -= 0.5
#     assert len(img) != 0, '[data] Loading image failed: {}'.format(img_file)
    img = tools.load_rgb(im_file=img_file, 
                        im_size=im_size, 
                        im_mode='CHW',
                        im_norm='none')

    data.append(img)
data=np.asarray(data)

/users/visics/kkelchte/tensorflow_1.8/lib/python2.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [6]:
network

'tinyv3_net'

In [4]:
# evaluate model (without autograd) and measure FPS
load_time=[]
predict_time=[]
for i in range(data.shape[0]-n_frames):
    if i%100 == 0: print("Frame {0}/{1}".format(i,data.shape[0]))
    stime=time.time()
#     if not ('nfc' in network or '3d' in network or 'LSTM' in network):
    if '3d' in network:
        imgs=list(data[i:i+n_frames])
        img=np.concatenate(imgs, axis=0)
    elif 'nfc' in network:
        img=data[i:i+n_frames]
    elif 'LSTM' in network:
        img=data[i:i+1]
    else:
        img=data[i]
    img=np.expand_dims(img,0)
#     print img.shape
#     sys.exit()
    
    inputs=torch.from_numpy(img).type(torch.FloatTensor).to(device)
    load_time.append(time.time()-stime)
    stime=time.time()
    if not 'LSTM' in network:
        predictions = net.forward(inputs, train=False)
    else:
        h_t,c_t=net.get_init_state(1)
        inputs=(inputs,(h_t.to(device), c_t.to(device)))
        predictions = net.forward(inputs, train=False)
    predict_time.append(time.time()-stime)
print("load: {0}({1}), prediction: {2}({3})".format(np.mean(load_time),
                                                  np.std(load_time),
                                                  np.mean(predict_time),
                                                  np.std(predict_time)))
print("speed: {0}".format(1/np.mean(predict_time)))

Frame 0/100
load: 0.000292868958306(0.000122613234886), prediction: 0.0179719900348(0.00430646301616)
speed: 55.6421408017


In [18]:
img=torch.from_numpy(np.expand_dims(data[0],0)).type(torch.FloatTensor).to(device)
for l in range(len(net.network.features)):
    print l, net.network.features[:l](img).shape
for l in range(len(net.network.classifier)):
    print l, net.network.classifier[:l](img).shape

 0 torch.Size([1, 3, 128, 128])
1 torch.Size([1, 10, 30, 30])
2 torch.Size([1, 10, 30, 30])
3 torch.Size([1, 20, 7, 7])
0 torch.Size([1, 3, 128, 128])
1

RuntimeError: size mismatch, m1: [384 x 128], m2: [980 x 512] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:940